In [2]:
%cd wwdrive2

/home/ubuntu/wwdrive2


In [3]:
!pip install --upgrade happytransformer transformers datasets
!pip install wandb
#!pip install numpy==1.24.3  # This is an example; choose any version within the compatible range


Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: flatbuffers 1.12.1-git20200711.33e2d80-dfsg1-0.6 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of flatbuffers or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: flatbuffers 1.12.1-git20200711.33e2d80-dfsg1-0.6 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of flatbuffers or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://git

In [ ]:
import os
os.environ["WANDB_API_KEY"] = "339e8cbc62b895edfe6f9f7476adf12a96dd3ed6"


In [ ]:
import wandb
wandb.login()
wandb.init(
    # set the wandb project where this run will be logged
    project="t5-mini-grammar",
)

In [ ]:
import pandas as pd

def remove_non_string_rows(df, columns):
    """
    Remove rows where any specified columns do not contain string entries.
    
    Parameters:
    - df: pandas DataFrame containing your data.
    - columns: list of column names to check for non-string entries.
    
    Returns:
    - The DataFrame with rows containing non-string entries in the specified columns removed.
    - A list of indices of the rows that were removed.
    """
    indices_to_remove = set()

    for column in columns:
        for i, entry in enumerate(df[column]):
            if not isinstance(entry, str):
                # Add the index of the non-string entry to the set of indices to remove
                indices_to_remove.add(i)
    
    # If there are any indices to remove, drop those rows
    if indices_to_remove:
        print(f"Removing rows with non-string entries in the following indices: {sorted(indices_to_remove)}")
        df = df.drop(index=list(indices_to_remove)).reset_index(drop=True)
    else:
        print("No non-string entries found in the specified columns.")
    
    return df, indices_to_remove

# Example usage
df = pd.read_csv("eval.csv")  # Load your dataset
columns_to_check = ['input', 'target']  # Specify the columns to check
df, indices_removed = remove_non_string_rows(df, columns_to_check)
# Optionally, save the cleaned DataFrame back to a CSV
df.to_csv("cleaned_eval.csv", index=False)

df = pd.read_csv("train.csv")  # Load your dataset
columns_to_check = ['input', 'target']  # Specify the columns to check
df, indices_removed = remove_non_string_rows(df, columns_to_check)
# Optionally, save the cleaned DataFrame back to a CSV
df.to_csv("cleaned_train.csv", index=False)



In [ ]:
from happytransformer import HappyTextToText, TTEvalArgs

happy_tt = HappyTextToText("T5", "t5-small")
before_result = happy_tt.eval("cleaned_eval.csv")
print("Before loss:", before_result.loss)


In [ ]:
from happytransformer import TTTrainArgs
# Define your parameters
params = TTTrainArgs(
    fp16=True,
    learning_rate=5e-5,
    num_train_epochs=3,
    batch_size=32,
    save_steps=1000  # Automatic checkpoint every 1000 steps
)

#model.save_pretrained("my_checkpoint")
#model = HappyTextToText.from_pretrained("my_checkpoint")

# Specify your training file path
train_file_path = "cleaned_train.csv"

In [ ]:
# Train the model with Happy Transformers
happy_tt.train(train_file_path, args=params)

In [13]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Assuming you have your model loaded
model_name = "willwade/t5-small-spoken-typo"
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)
#tokenizer = T5Tokenizer.from_pretrained(model_name, legacy=False)

# Save the model and tokenizer
model_save_path = "model/t5-small-spoken-typo"
tokenizer_save_path = "model/t5-small-spoken-typo"

model.save_pretrained(model_save_path)
tokenizer.save_pretrained(tokenizer_save_path)

# To load the model and tokenizer later
model = T5ForConditionalGeneration.from_pretrained(model_save_path)
tokenizer = T5Tokenizer.from_pretrained(tokenizer_save_path)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
before_loss = happy_tt.eval("cleaned_eval.csv")
print("After loss: ", before_loss.loss)

In [15]:

import torch
from transformers import T5ForConditionalGeneration

# Load your pre-trained model
model = T5ForConditionalGeneration.from_pretrained("./model/t5-small-spoken-typo")

# Apply dynamic quantization
quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)

# Attempt to save the quantized model
quantized_model.save_pretrained("./model/quantized-t5-small-spoken-typo")


AttributeError: 'torch.dtype' object has no attribute 'device'

In [10]:
# test it!
from happytransformer import HappyGeneration, TTSettings

# Load the pre-quantized model
happy_tt_pre = HappyGeneration(model_name="./model/t5-small-spoken-typo")

# Define beam search settings
beam_settings = TTSettings(num_beams=5, min_length=1, max_length=20)

# Generate text with the pre-quantized model
example_1 = "grammar: This sentences, has bads grammar and spelling!"
result_1_pre = happy_tt_pre.generate_text(example_1, args=beam_settings)
print("Pre-Quantized Model Result 1:", result_1_pre.text)

example_2 = "grammar: I am enjoys, writtings articles ons AI."
result_2_pre = happy_tt_pre.generate_text(example_2, args=beam_settings)
print("Pre-Quantized Model Result 2:", result_2_pre.text)

# Load the post-quantized model
# Ensure you adjust the path to where your quantized model is saved
happy_tt_post = HappyGeneration(load_path="./model/quantized-t5-small-spoken-typo")

# Generate text with the post-quantized model
result_1_post = happy_tt_post.generate_text(example_1, args=beam_settings)
print("Post-Quantized Model Result 1:", result_1_post.text)

result_2_post = happy_tt_post.generate_text(example_2, args=beam_settings)
print("Post-Quantized Model Result 2:", result_2_post.text)


ValueError: Unrecognized configuration class <class 'transformers.models.t5.configuration_t5.T5Config'> for this kind of AutoModel: AutoModelForCausalLM.
Model type should be one of BartConfig, BertConfig, BertGenerationConfig, BigBirdConfig, BigBirdPegasusConfig, BioGptConfig, BlenderbotConfig, BlenderbotSmallConfig, BloomConfig, CamembertConfig, LlamaConfig, CodeGenConfig, CpmAntConfig, CTRLConfig, Data2VecTextConfig, ElectraConfig, ErnieConfig, FalconConfig, FuyuConfig, GitConfig, GPT2Config, GPT2Config, GPTBigCodeConfig, GPTNeoConfig, GPTNeoXConfig, GPTNeoXJapaneseConfig, GPTJConfig, LlamaConfig, MarianConfig, MBartConfig, MegaConfig, MegatronBertConfig, MistralConfig, MixtralConfig, MptConfig, MusicgenConfig, MvpConfig, OpenLlamaConfig, OpenAIGPTConfig, OPTConfig, PegasusConfig, PersimmonConfig, PhiConfig, PLBartConfig, ProphetNetConfig, QDQBertConfig, Qwen2Config, ReformerConfig, RemBertConfig, RobertaConfig, RobertaPreLayerNormConfig, RoCBertConfig, RoFormerConfig, RwkvConfig, S

In [14]:
import shutil

# Define the directory containing your model and the output zip file name
model_directory = "./model/t5-small-spoken-typo"
output_zip_file = "./model/t5-small-spoken-typo.zip"

# Create a zip archive from the model directory
shutil.make_archive(base_name=output_zip_file[:-4], format='zip', root_dir=model_directory)


'/home/ubuntu/wwdrive2/model/t5-small-spoken-typo.zip'